# Project 6: VCFAD - Voice Cloning and Fake Audio Detection
(TODO)

### Data Description
There are two datasets we will use for this project:

**TIMIT Dataset**

This dataset contains 6300 sentences (10 sentences spoken by 630 different speakers) and will be used to create a voice cloner that generates fake samples.

**CommonVoice Dataset**

This dataset contains ... is used for our real voice samples.

### Project Setup

In [28]:
!mkdir "data"
!mkdir "data/real"
!mkdir "data/fake"
!mkdir "data/fake_by_speaker"
!mkdir "data/fake_by_speaker/0"
!mkdir "data/fake_by_speaker/1"
!mkdir "data/fake_by_speaker/2"
!mkdir "data/fake_by_speaker/3"
!mkdir "data/fake_by_speaker/4"
!mkdir "data/fake_by_speaker/5"
#!mkdir "data/me_real"
#!mkdir "data/me_fake"

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/real’: File exists
mkdir: cannot create directory ‘data/fake’: File exists
mkdir: cannot create directory ‘data/fake_by_speaker’: File exists
mkdir: cannot create directory ‘data/fake_by_speaker/0’: File exists
mkdir: cannot create directory ‘data/fake_by_speaker/1’: File exists
mkdir: cannot create directory ‘data/fake_by_speaker/2’: File exists
mkdir: cannot create directory ‘data/fake_by_speaker/3’: File exists
mkdir: cannot create directory ‘data/fake_by_speaker/4’: File exists
mkdir: cannot create directory ‘data/fake_by_speaker/5’: File exists


In [20]:
# Install the necessary packages
#!pip install gtts
!pip install pydub
!pip install TTS
!pip3 install numpy==1.21.0
!pip install SpeechRecognition
!pip install --upgrade protobuf
!pip install tensorflow==2.11.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-4.22.3-cp37-abi3-manylinux2014_x86_64.whl (302 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trainer 0.0.20 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.3 which is incompa

In [21]:
# Import the necessary packages
import pandas as pd
from IPython.display import Audio
import subprocess
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import randint
from TTS.api import TTS
from IPython import display
from tensorflow.keras import layers
from tensorflow.keras import models
import speech_recognition as sr
from os import path
from pydub import AudioSegment
from imutils import paths

In [22]:
# Mount Google Drive and import datasets
from google.colab import drive
import zipfile
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/Datasets/archive.zip", 'r')
zip_ref.extractall("/content/timit")
zip_ref.close()

In [24]:
zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/Datasets/cv-corpus-12.0-delta-2022-12-07.zip", 'r')
zip_ref.extractall("/content/cv")
zip_ref.close()

In [25]:
zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/Datasets/cv-corpus-10.0-delta-2022-07-04.zip", 'r')
zip_ref.extractall("/content/cv")
zip_ref.close()

In [26]:
zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/Datasets/cv-corpus-11.0-delta-2022-09-21.zip", 'r')
zip_ref.extractall("/content/cv")
zip_ref.close()

In [27]:
cv_paths = list(paths.list_files("/content/cv/", ".mp3"))
i=0
# For each file in TRAIN
for path in cv_paths:
  subprocess.check_call(['ffmpeg', '-i', f'{path}', '-acodec', 'pcm_s16le', '-ar' ,'16000', '-ac', '1', f'data/real/{i}.wav'])
  i += 1
  if i > 1500:
    break;

CalledProcessError: ignored

### Data Exploration

For the Common Voice dataset, we'll use the Delta Segment 12.0. And from that, we'll use only the validated clips.

In [29]:
df = pd.read_csv('/content/cv/cv-corpus-12.0-delta-2022-12-07/en/validated.tsv', sep='\t', header=0)

print(f"Total samples: {len(df)}")

Total samples: 6717


There are 6717 samples in this dataset. Now lets check if these are unique sentences.

In [30]:
df.sentence.value_counts()

Training opportunities occur every February and March.                                     3
The house itself was expanded twice during the nineteenth century.                         2
She was the Associations's first Secretary.                                                2
Especially tall fly towers pose a balance problem for standard counterweight line sets.    2
In this film she portrayed a magazine illustrator seeking western types.                   2
                                                                                          ..
This was enough to win the Olympic bronze medal.                                           1
He was survived by his four children Frances, Helen, Joseph and Veronica.                  1
We hope this campaign will help us achieve that.                                           1
I have it still.                                                                           1
The station was originally named Palmers Green and Southgate.         

There are multiple with non-unique sentences. Lets reduce our DataFrame so only unique sentences remain.

In [31]:
df = df.drop_duplicates("sentence")
print(f"Total samples: {len(df)}")

Total samples: 6675


There are now 6675 samples remaining. Lets take the first 1000 with unique sentences and convert each clip to .wav format.

In [32]:
df = df.head(1000)




In [33]:
# Convert clips to wav format
for path in df.path:
  subprocess.check_call(['ffmpeg', '-i', f'/content/cv/cv-corpus-12.0-delta-2022-12-07/en/clips/{path}', '-acodec', 'pcm_s16le', '-ar' ,'16000', '-ac', '1', f'data/real/{path}.wav'])

CalledProcessError: ignored

In [ ]:
# Calculate duration for each clip
import os
from pydub import AudioSegment

# specify the folder path containing audio files
folder_path = '/content/data/real'

# get all audio files in the folder
audio_files = [f for f in os.listdir(folder_path) if f.endswith(('.wav'))]

# iterate over the audio files and get their duration
for audio_file in audio_files:
    audio = AudioSegment.from_file(os.path.join(folder_path, audio_file))
    duration = len(audio) / 1000.0
    df.loc[df['path'] == audio_file[:-4], 'duration'] = duration
    #print(f"{audio_file[:-4]}: {duration:.2f} seconds")


In [ ]:
# Plot a histogram for the duration of the real audio files
import plotly.express as px
#df = px.data.tips()
fig = px.histogram(df, x="duration")
fig.show()

### Build a voice cloning system

#### Strategy 1: Use text to speech library to generate 'fake' voice clips

In [34]:
# List available 🐸TTS models and choose the first one
model_name = TTS.list_models()[0]

# Init TTS
tts = TTS(model_name)

No API token found for 🐸Coqui Studio voices - https://coqui.ai 
Visit 🔗https://app.coqui.ai/account to get one.
Set it as an environment variable `export COQUI_STUDIO_TOKEN=<token>`

 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256

In [35]:
seed(1)
i=0
for sentence in df.sentence:
  speaker = randint(0, 5)
  tts.tts_to_file(text=sentence, speaker=tts.speakers[speaker], language=tts.languages[0], file_path=f"data/fake/{i}.wav")
  tts.tts_to_file(text=sentence, speaker=tts.speakers[speaker], language=tts.languages[0], file_path=f"data/fake/{speaker}/{i}.wav")
  
  # Save fake path
  df.loc[i,'fake_path'] = f"data/fake/{i}.wav"
  #df.loc[i,'fake_path'] = f"data/fake/{speaker}/{i}.wav"

  i += 1

 > Text splitted to sentences.
['He was commonly referred to as the "Blacksmith of Ballinalee".']
 > Processing time: 1.9502630233764648
 > Real-time factor: 0.49942715067259025
 > Text splitted to sentences.
['He was commonly referred to as the "Blacksmith of Ballinalee".']
 > Processing time: 1.339958906173706
 > Real-time factor: 0.3335720453506861


FileNotFoundError: ignored

In [ ]:
Audio('/content/data/fake/844.wav', autoplay=True) 

In [ ]:
Audio('/content/data/fake/845.wav', autoplay=True) 

In [ ]:
Audio('/content/data/fake/846.wav', autoplay=True) 

In [ ]:
# Calculate duration for each clip

# specify the folder path containing audio files
folder_path = '/content/data/fake'

# get all audio files in the folder
audio_files = [f for f in os.listdir(folder_path) if f.endswith(('.wav'))]

# iterate over the audio files and get their duration
durations = []
for audio_file in audio_files:
    audio = AudioSegment.from_file(os.path.join(folder_path, audio_file))
    duration = len(audio) / 1000.0
    #df.loc[df['path'] == audio_file[:-4], 'duration'] = duration
    #print(f"{audio_file[:-4]}: {duration:.2f} seconds")
    durations.append(duration)

In [ ]:
# Plot a histogram for the duration of the fake audio files
import plotly.express as px
#df = px.data.tips()
fig = px.histogram(durations).update_layout(
    xaxis_title="Duration"
)
fig.show()

##### Calculate word error rate
- Audio transcription
- Match with original text (each word)
- Calculate error

In [ ]:
def wer(ref, hyp ,debug=True):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#cor " + str(numCor))
        print("#sub " + str(numSub))
        print("#del " + str(numDel))
        print("#ins " + str(numIns))
    # return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'numCor':numCor, 'numSub':numSub, 'numIns':numIns, 'numDel':numDel, "numCount": len(r)}

In [ ]:
# Calculate word error rate for all sentences
i=0
r = sr.Recognizer()
for sentence in df.sentence:  
  # Transcribe fake audio                                       
  with sr.AudioFile(f"data/fake/{i}.wav") as source:
    audio = r.record(source)  # read the entire audio file
    try:
      df.loc[i,'fake_transcription'] = r.recognize_google(audio)
    except:
      df.loc[i,'fake_transcription'] = ''

  # Calculate word error rate  
  df.loc[i, 'wer'] = wer(sentence, df.loc[i, 'fake_transcription'], debug=False)['WER']

  i += 1

In [ ]:
print(f"The average word error rate (WER) is {df.loc[:, 'wer'].mean()}")

##### Speaker classification accuracy

In [ ]:
data_dir = '/content/data/fake'
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=64000, # Set the length of the output to be 4 seconds
    subset='both')

label_names = np.array(train_ds.class_names)

print("\nlabel names:", label_names)

In [ ]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

In [ ]:
# Break up val_ds into a validation and test set
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)

In [ ]:
for example_audio, example_labels in train_ds.take(1):  
  print(example_audio.shape)
  print(example_labels.shape)

In [ ]:
# Convert waveforms to spectograms
def get_spectrogram(waveform):
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

In [ ]:
for i in range(6):
  label = label_names[example_labels[i]]
  waveform = example_audio[i]
  spectrogram = get_spectrogram(waveform)

  print('Label:', label)
  print('Waveform shape:', waveform.shape)
  print('Spectrogram shape:', spectrogram.shape)
  print('Audio playback')
  display.display(display.Audio(waveform, rate=16000))

In [ ]:
def plot_spectrogram(spectrogram, ax):
  if len(spectrogram.shape) > 2:
    assert len(spectrogram.shape) == 3
    spectrogram = np.squeeze(spectrogram, axis=-1)
  # Convert the frequencies to log scale and transpose, so that the time is
  # represented on the x-axis (columns).
  # Add an epsilon to avoid taking a log of zero.
  log_spec = np.log(spectrogram.T + np.finfo(float).eps)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)

Now create spectrogram datasets from the audio datasets

In [ ]:
def make_spec_ds(ds):
  return ds.map(
      map_func=lambda audio,label: (get_spectrogram(audio), label),
      num_parallel_calls=tf.data.AUTOTUNE)
  
train_spectrogram_ds = make_spec_ds(train_ds)
val_spectrogram_ds = make_spec_ds(val_ds)
test_spectrogram_ds = make_spec_ds(test_ds)

for example_spectrograms, example_spect_labels in train_spectrogram_ds.take(1):
  break

rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(16, 9))

for i in range(n):
    r = i // cols
    c = i % cols
    ax = axes[r][c]
    plot_spectrogram(example_spectrograms[i].numpy(), ax)
    ax.set_title(label_names[example_spect_labels[i].numpy()])

plt.show()

Now lets train the model

In [ ]:
train_spectrogram_ds = train_spectrogram_ds.cache().shuffle(10000).prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = val_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = test_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)

input_shape = example_spectrograms.shape[1:]
print('Input shape:', input_shape)
num_labels = len(label_names)

# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
# Fit the state of the layer to the spectrograms
# with `Normalization.adapt`.
norm_layer.adapt(data=train_spectrogram_ds.map(map_func=lambda spec, label: spec))

model = models.Sequential([
    layers.Input(shape=input_shape),
    # Downsample the input.
    layers.Resizing(32, 32),
    # Normalize.
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
EPOCHS = 10
history = model.fit(
    train_spectrogram_ds,
    validation_data=val_spectrogram_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

In [ ]:
%matplotlib inline
metrics = history.history
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch')
plt.ylabel('Loss [CrossEntropy]')

plt.subplot(1,2,2)
plt.plot(history.epoch, 100*np.array(metrics['accuracy']), 100*np.array(metrics['val_accuracy']))
plt.legend(['accuracy', 'val_accuracy'])
plt.ylim([0, 100])
plt.xlabel('Epoch')
plt.ylabel('Accuracy [%]')

plt.show()

In [ ]:
model.evaluate(test_spectrogram_ds, return_dict=True, verbose=True)
y_pred = model.predict(test_spectrogram_ds)
y_true = tf.concat(list(test_spectrogram_ds.map(lambda s,lab: lab)), axis=0)
y_labels = np.concatenate([y for x, y in test_spectrogram_ds], axis=0)

from sklearn.metrics import classification_report

y_pred = model.predict(test_spectrogram_ds)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_labels, y_pred_bool))

#### Strategy 2: Voice cloning with YourTTS

In [ ]:
# List available 🐸TTS models and choose the first one
model_name = TTS.list_models()[0]

# Init TTS
tts = TTS(model_name)

In [ ]:
train_paths = list(paths.list_files("/content/timit/archive/data/TRAIN/", ".wav", ".WAV.wav"))
i=0
# For each file in TRAIN
for path in train_paths:
  tts.tts_to_file(df.sample()['sentence'].values[0], speaker_wav=path, language="en", file_path=f"data/fake/{i}.wav")

  i += 1

In [ ]:
print(f'There are {len(list(paths.list_files("/content/data/fake", ".wav")))} generated fake samples.')

In [ ]:
Audio('output.wav', autoplay=True) 

#### Strategy 3: Use TIMIT Dataset to clone voices

In [ ]:
# Implement the WaveNet model

In [ ]:
def generate_fake(source_speaker, target_speaker, text) ?

In [ ]:
# Collect 1600 positive samples (from the CommonVoice dataset)


# Generate negative samples (from voice cloning system)


# Train a model to identify real and fake audio

### Build a fake audio detection system



We can load the data using `keras.utils.audio_dataset_from_directory`.

In [ ]:
data_dir = '/content/data'
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=64000, # Set the length of the output to be 4 seconds
    subset='both')

label_names = np.array(train_ds.class_names)

print("\nlabel names:", label_names)

In [ ]:
train_ds.element_spec

This dataset only contains single channel audio, so use the `tf.squeeze` function to drop the extra axis:

In [ ]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

In [ ]:
# Break up val_ds into a validation and test set
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)

In [ ]:
for example_audio, example_labels in train_ds.take(1):  
  print(example_audio.shape)
  print(example_labels.shape)

In [ ]:
# Convert waveforms to spectograms
def get_spectrogram(waveform):
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

In [ ]:
label_names

In [ ]:
for i in range(6):
  label = label_names[example_labels[i]]
  waveform = example_audio[i]
  spectrogram = get_spectrogram(waveform)

  print('Label:', label)
  print('Waveform shape:', waveform.shape)
  print('Spectrogram shape:', spectrogram.shape)
  print('Audio playback')
  display.display(display.Audio(waveform, rate=16000))

In [ ]:
def plot_spectrogram(spectrogram, ax):
  if len(spectrogram.shape) > 2:
    assert len(spectrogram.shape) == 3
    spectrogram = np.squeeze(spectrogram, axis=-1)
  # Convert the frequencies to log scale and transpose, so that the time is
  # represented on the x-axis (columns).
  # Add an epsilon to avoid taking a log of zero.
  log_spec = np.log(spectrogram.T + np.finfo(float).eps)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)

Now create spectrogram datasets from the audio datasets

In [ ]:
def make_spec_ds(ds):
  return ds.map(
      map_func=lambda audio,label: (get_spectrogram(audio), label),
      num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
train_spectrogram_ds = make_spec_ds(train_ds)
val_spectrogram_ds = make_spec_ds(val_ds)
test_spectrogram_ds = make_spec_ds(test_ds)

In [ ]:
for example_spectrograms, example_spect_labels in train_spectrogram_ds.take(1):
  break

In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(16, 9))

for i in range(n):
    r = i // cols
    c = i % cols
    ax = axes[r][c]
    plot_spectrogram(example_spectrograms[i].numpy(), ax)
    ax.set_title(label_names[example_spect_labels[i].numpy()])

plt.show()

Now lets build and train the model

In [ ]:
train_spectrogram_ds = train_spectrogram_ds.cache().shuffle(10000).prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = val_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = test_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
input_shape = example_spectrograms.shape[1:]
print('Input shape:', input_shape)
num_labels = len(label_names)

# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
# Fit the state of the layer to the spectrograms
# with `Normalization.adapt`.
norm_layer.adapt(data=train_spectrogram_ds.map(map_func=lambda spec, label: spec))

model = models.Sequential([
    layers.Input(shape=input_shape),
    # Downsample the input.
    layers.Resizing(32, 32),
    # Normalize.
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
EPOCHS = 10
history = model.fit(
    train_spectrogram_ds,
    validation_data=val_spectrogram_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

In [ ]:
metrics = history.history
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch')
plt.ylabel('Loss [CrossEntropy]')

plt.subplot(1,2,2)
plt.plot(history.epoch, 100*np.array(metrics['accuracy']), 100*np.array(metrics['val_accuracy']))
plt.legend(['accuracy', 'val_accuracy'])
plt.ylim([0, 100])
plt.xlabel('Epoch')
plt.ylabel('Accuracy [%]')

plt.show()

In [ ]:
model.evaluate(test_spectrogram_ds, return_dict=True, verbose=True)

In [ ]:
y_pred = model.predict(test_spectrogram_ds)

In [ ]:
y_true = tf.concat(list(test_spectrogram_ds.map(lambda s,lab: lab)), axis=0)

#### Results

In [ ]:
y_labels = np.concatenate([y for x, y in test_spectrogram_ds], axis=0)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_spectrogram_ds)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_labels, y_pred_bool))


In [ ]:
print(y_labels)

In [ ]:
np.unique(y_labels, return_counts=True)

In [ ]:
y_pred_val = model.predict(test_spectrogram_ds)
y_pred_val_bool = np.argmax(y_pred_val, axis=1)

In [ ]:
np.argmax(model.predict(val_spectrogram_ds), axis=1)

In [ ]:
np.argmax(model.predict(test_spectrogram_ds), axis=1)

### Run inference with my voice

In [ ]:
list(paths.list_files("/content/data/me", ".wav"))

In [ ]:
# TODO: Record an audio (not fake) and do an inference with the model
my_paths = list(paths.list_files("/content/data/me", ".wav"))

for path in my_paths:
  x = tf.io.read_file(str(path))
  x, sample_rate = tf.audio.decode_wav(x, desired_samples=64000,)
  x = tf.squeeze(x, axis=-1)
  waveform = x
  x = get_spectrogram(x)
  x = x[tf.newaxis,...]

  %matplotlib inline
  prediction = model(x)
  x_labels = ['fake', 'real']
  plt.bar(x_labels, tf.nn.softmax(prediction[0]))
  plt.title('Voice Clip Prediction')
  plt.show()

  display.display(display.Audio(waveform, rate=16000))

Now lets use the voice cloner to generate fake samples of my voice.

In [ ]:
real_voice_paths = list(paths.list_files("/content/data/me", ".wav"))

i=0
for path in real_voice_paths:
  tts.tts_to_file(df.sample()['sentence'].values[0], speaker_wav=path, language="en", file_path=f"data/me_fake/{i}.wav")
  i += 1

In [ ]:
my_paths = list(paths.list_files("/content/data/me_fake", ".wav"))

for path in my_paths:
  x = tf.io.read_file(str(path))
  x, sample_rate = tf.audio.decode_wav(x, desired_samples=64000,)
  x = tf.squeeze(x, axis=-1)
  waveform = x
  x = get_spectrogram(x)
  x = x[tf.newaxis,...]

  %matplotlib inline
  prediction = model(x)
  x_labels = ['fake', 'real']
  plt.bar(x_labels, tf.nn.softmax(prediction[0]))
  plt.title('Voice Clip Prediction')
  plt.show()

  display.display(display.Audio(waveform, rate=16000))

### Summary
(TODO)
Further considerations:
- Real world samples contain background noise